In [1]:
# Import Relevant Packages
# Code is credited to Dr. Michelle McCrystall of the University of Manitoba

import numpy as np
import iris as iris
import iris.coord_categorisation
import scipy as sp
from scipy import signal
import glob
from iris.experimental.equalise_cubes import equalise_attributes
import pandas as pd

import cartopy.crs as ccrs
import iris.plot as iplt
import matplotlib.pyplot as plt
import sys

import cartopy
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import warnings
import matplotlib.gridspec as gridspec
import iris.quickplot as qplt
import matplotlib
import matplotlib.ticker as mticker
import iris.quickplot as qplt

import matplotlib.path as mpath
import cf_units
from iris.experimental.regrid import regrid_weighted_curvilinear_to_rectilinear
from dateutil import parser
from datetime import datetime, timedelta
from netCDF4 import Dataset, date2num, num2date
from datetime import datetime, timedelta
import matplotlib as mpl
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")


In [2]:
# Make a dummy grid and input desired 1D grid coordinates

def make_grid(lat_values, lon_values):
            
    latitude = iris.coords.DimCoord(lat_values,
                                    standard_name='latitude',
                                    units='degrees_north',
                                    coord_system=iris.coord_systems.GeogCS(iris.fileformats.pp.EARTH_RADIUS))
    longitude = iris.coords.DimCoord(lon_values,                     
                                     standard_name='longitude',
                                     units='degrees_east',
                                     coord_system=iris.coord_systems.GeogCS(iris.fileformats.pp.EARTH_RADIUS))

    dummy_data = np.zeros((len(lat_values), len(lon_values)), dtype=np.float32)
    new_cube = iris.cube.Cube(dummy_data, dim_coords_and_dims=[(latitude, 0), (longitude, 1)])
    
    return new_cube

In [ ]:
# Import File
file1 = #path to file


In [3]:

regrid_lat = np.linspace(60, 90, 16)
regrid_lon = np.linspace(0, 359, 180)



In [27]:
cube_test = iris.load_cube(file1)

# Use constraint to reduce size of data proccessed if necessary
year_test = iris.Constraint(time=lambda cell: 2090 <= cell.point.year <= 2100)
cube = cube_test.extract(year_test)

In [28]:
ntime = cube.coord('time').points

if len(cube.coords('year'))==0:
	iris.coord_categorisation.add_year(cube, 'time', name='year')
    
# Find start year, end year
startyear = np.str(cube.coord('year')[0].points[0]) 
endyear = np.str(cube.coord('year')[-1].points[0]) 

new_cube = cube

# Cube weights
weights = np.ones(new_cube[0].shape)
print(weights)

# Create a cube to regird onto
target_grid_cube = make_grid(regrid_lat, regrid_lon)

# Guess bounds for lat and lon
target_grid_cube.coord('longitude').guess_bounds()
target_grid_cube.coord('latitude').guess_bounds()

# Force coordinate system onto axis
new_cube.coord(axis='x').coord_system = iris.coord_systems.GeogCS(iris.fileformats.pp.EARTH_RADIUS)
new_cube.coord(axis='y').coord_system = iris.coord_systems.GeogCS(iris.fileformats.pp.EARTH_RADIUS)

regridded_cube_per_model = iris.cube.CubeList([])

# Loop through each time series 
for i in range(0, len(ntime)):
    
    # Regrid different grids
    regridded_cube= regrid_weighted_curvilinear_to_rectilinear(new_cube[i,:,:], weights, target_grid_cube)
    
    cast_data = regridded_cube.data.astype(np.float32)
    regridded_cube.data = cast_data
    
    regridded_cube_per_model.append(regridded_cube)

cube_regridded = regridded_cube_per_model.merge_cube()

[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]


In [29]:
# Check regridded cube is 1D array with correct coordinates
print(cube_regridded)

sea_ice_thickness / (m)             (time: 132; latitude: 16; longitude: 180)
     Dimension coordinates:
          time                           x              -              -
          latitude                       -              x              -
          longitude                      -              -              x
     Auxiliary coordinates:
          year                           x              -              -
     Attributes:
          Conventions: CF-1.7 CMIP-6.2
          activity_id: ScenarioMIP
          branch_method: standard
          branch_time_in_child: 59400.0
          branch_time_in_parent: 59400.0
          cmor_version: 3.4.0
          comment: Actual (floe) thickness of sea ice (NOT volume divided by grid area as...
          creation_date: 2020-05-05T09:49:39Z
          cv_version: 6.2.37.5
          data_specs_version: 01.00.29
          experiment: update of RCP8.5 based on SSP5
          experiment_id: ssp585
          external_variables: areacello
    

In [30]:
# Save regridded cube
iris.save(cube_regridded,r'C:\Users\Asus\Desktop\Masters Project\Uncut Data\sithick 10 May\regrid 10 may (16x180)\sithick_HadGEM3-GC31-MM-ssp585.nc')